Using QuantLib

In [ ]:
import QuantLib as ql

# ✅ Ensure evaluation date is set correctly
today = ql.Date().todaysDate()
ql.Settings.instance().evaluationDate = today

print(f"Evaluation Date (today): {today}")

# ✅ Extend the market zero rates to ensure curve extends past swaption expiry
zc = {
    'jours': [30, 60, 90, 180, 360, 720, 1080, 1440, 1800, 2160, 2520, 2880],  # Extended maturities (8 years)
    'taux': [0.01, 0.015, 0.017, 0.02, 0.022, 0.025, 0.026, 0.027, 0.028, 0.029, 0.030, 0.031]
}

spot_dates = [today + ql.Period(i, ql.Days) for i in zc['jours']]
spot_rates = zc['taux']

# ✅ Define yield curve parameters
day_count = ql.Thirty360(ql.Thirty360.BondBasis)
calendar = ql.NullCalendar()
interpolation = ql.Linear()
compounding = ql.Compounded
compounding_frequency = ql.Annual

# ✅ Fix: Ensure yield curve starts at `today`
spot_curve = ql.ZeroCurve(
    [today] + spot_dates,  # ✅ Ensure first date is today
    [spot_rates[0]] + spot_rates,  # ✅ Ensure first rate matches
    day_count,
    calendar,
    interpolation,
    compounding,
    compounding_frequency
)
spot_curve.enableExtrapolation()  # ✅ Ensure rates are available beyond last maturity

yts = ql.YieldTermStructureHandle(spot_curve)

# ✅ Check reference date again
print(f"Zero Curve Reference Date (after fix): {yts.referenceDate()}")

# ✅ Define Hull-White Model
model = ql.HullWhite(yts)

# ✅ Set up the pricing engine
engine = ql.JamshidianSwaptionEngine(model, yts)

# ✅ Define the underlying swap
nominal = 1_000_000
fixed_rate = 0.02
fixed_schedule = ql.Schedule(
    today, ql.Date(12, 11, 2030), ql.Period(ql.Annual), 
    calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, 
    ql.DateGeneration.Forward, False
)
floating_schedule = fixed_schedule
ibor_index = ql.Euribor6M(yts)

underlying_swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, 
    nominal,  
    fixed_schedule, 
    fixed_rate, 
    day_count,
    floating_schedule, 
    ibor_index, 
    0.0,  
    day_count
)

# ✅ Fix: Ensure exercise date is in the future
exercise_date = ql.Date(12, 11, 2025)
if exercise_date <= today:
    raise ValueError(f"Exercise date {exercise_date} must be after evaluation date {today}.")

exercise = ql.EuropeanExercise(exercise_date)
swaption = ql.Swaption(underlying_swap, exercise)

# ✅ Set the correct pricing engine
swaption.setPricingEngine(engine)

# ✅ Compute NPV
npv = swaption.NPV()
print(f"Swaption NPV: {npv}")


Own code